In [3]:
import pandas as pd
import seaborn as sb
import csv
from sympy import FiniteSet

import pysam
import os
import math
import random
import numpy as np

from pathlib import Path
from random import seed
from random import randint

In [2]:
#funciones

def powerset(fullset):
  listsub = list(fullset)
  subsets = []
  for i in range(2**len(listsub)):
    subset = []
    for k in range(len(listsub)):            
      if i & 1<<k:
        subset.append(listsub[k])
    subsets.append(subset)        
  return subsets

Vamos a mapear los reads verdaderos de maiz a los genomas de la base de datos interna de referencia de _Clavibacter_ (BDIRC) 
0. Crear link simbolicos
1. Indexar los genomas de BDIRC con bowtie
2. Mapear con bowtie a esos genomas
3. Si queremos visualizar los reads podemos usar tablet, y necesitamos samtools para indexar el genoma y ver los reads alineados.
https://www.metagenomics.wiki/tools/bowtie2

In [3]:
%%bash 
source /opt/anaconda3/etc/profile.d/conda.sh
conda activate metagenomics

A continuación creamos indices con para cada uno de los genomas refencia, para ello utilizamos el comando `bowtie2-build`. Los respectivos archivos de salida los almacenamos en la carpeta `index-bowtie`. Adicionalmente, creamos el archivo `list-genomes.txt` que contiene los nombre de cada archivo de salida del `bowtie2-build`.

In [ ]:

%%bash 
source /opt/anaconda3/etc/profile.d/conda.sh
conda activate metagenomics
ls genomas-clavi/clavi-genomes/*fna | while read line; 
do 
    id=$(echo $line| rev| cut -d'_' -f1,2,3 |rev |cut -d'.' -f1);
    bowtie2-build $line genomas-clavi/clavi-genomes/index-bowtie/$id;
    echo "$id" >>list-genomes.txt
done

#Por ahora solo corre en bash




Lo siguiente es mapear los reads a los genomas de referencia, utilizamos `bowtie2` y obtenemos arhivos de salidas *.sam* que guardaremos en la carpeta `output-bowtie` 

#reference=subsp_capsici_1101  ; usar para extraer las lineas de archivo sed '1q;d' index-genomes.txt
for infile in ex-reads-clavi/fastq/fastq/capsicum/*-clav-1.fq
 do 
base=$(basename ${infile} -clav-1.fq)
dir=$(dirname ${infile} )
bowtie2 -x genomas-clavi/clavi-genomes/index-bowtie/subsp_capsici_1101 -1 ${dir}/${base}-clav-1.fq -2 ${dir}/${base}-clav-2.fq  --no-unal -p 12 -S genomas-clavi/clavi-genomes/out-bowtie/${base}.sam
done

In [ ]:
%%bash 
source /opt/anaconda3/etc/profile.d/conda.sh
conda activate metagenomics
bash 01-mapear-reads-a-referencia.sh insidiosus_CFBP_2404

Ahora corremos el scrip `01-mapear-reads-a-referencia.sh` por cada una de la lista de genomas que se encuentra en `index-genomes.txt´ 

In [ ]:
cat index-genomes.txt | while read line;
do
    bash 01-mapear-reads-a-referencia.sh $line
done

## Prueba 1: Utilizar solo 3 genomas
Ahora procedor hacer lo anterior son con 3 genomas, 1 de Clavibacter Michiganensis Michiganensis (cmm), y 2 de Michagenesis capsisi. 
Primero cremos los index



In [ ]:
%%bash
source /opt/anaconda3/etc/profile.d/conda.sh
conda activate metagenomics
ls genomas-clavi/clavi-genomes/genomas-prueba1/*fna | while read line; 
do 
    id=$(echo $line| rev| cut -d'_' -f1,2,3 |rev |cut -d'.' -f1);
    bowtie2-build $line genomas-clavi/clavi-genomes/genomas-prueba1/index-bowtie/$id;
    echo "$id" >>list-genomasrefencia.txt
done

Ahora, hacemos el mapeo de los read simulados a cada genoma de refencia previamente indexado


In [ ]:
%%bash
source /opt/anaconda3/etc/profile.d/conda.sh
conda activate metagenomics

cat list-genomasrefencia.txt | while read line;
do
bowtie2 -x output-tda/prueba-3genomas/index-bowtie/$line -U reads-sim1K.fastq.fastq --no-unal -p 12 -S output-tda/prueba1Kread/out-bowtie/Simulados-${line}.sam
done

### Crear archivos .bam para visualizar y extraer información del match

In [6]:
%%bash
for infile in output-tda/prueba1Kread/out-bowtie/*.sam
do 
	base=$(basename ${infile} .sam)
	samtools view ${infile} -o output-tda/prueba1Kread/bam/${base}.bam
    samtools sort ${infile} -o output-tda/prueba1Kread/sorted/${base}_sorted.sam
done

Crea un archivo *.txt con la salida mostrando el identificador y el read que identifico en el genoma referencia.

In [51]:
%%bash
for infile in output-tda/prueba1Kread/bam/*.bam
do 
	base=$(basename ${infile} .bam)
	samtools sort ${infile} -o output-tda/prueba1Kread/sorted/${base}_sorted.bam
	samtools view ${infile} > output-tda/prueba1Kread/bam/${base}.txt
done

Obtener las columnas que nos interesan de los archivos .sam

In [52]:
%%bash
cd ~/GIT/clavibacter/scripts-tda/
ls output-tda/prueba1Kread/bam/*.txt | while read line
do
    base=$(basename ${line} .txt)
    cut -f 1-6 $line > output-tda/prueba1Kread/tablas/${base}_extrac.txt
done


Importar tablas de mapeos


In [3]:
ruta_tablas = '/home/shaday/GIT/clavibacter/scripts-tda/output-tda/prueba1Kread/tablas/'
contenido = os.listdir(ruta_tablas)
genomes = []
df=[]
for i in contenido:
    dftt = pd.read_csv(ruta_tablas + i, sep='\t', header=None) #importar las tablas para cada genoma
    dftt['Organims']=i[16:-11] #agregar el nombre del organismo
    df.append(dftt) #agregar al dataframe
    genomes.append(i[16:-11])
    
df=pd.concat(df)
df.columns= ['Qname','Flag', 'Rname', 'Pos','MapQ','CIGAR','Organims'] #cambiar nombre a las columnas del dataframe
df 

,Qname,Flag,Rname,Pos,MapQ,CIGAR,Organims
0,NC_009478.1_0,0,NZ_CP048047.1,578115,42,150M,capsici_1106
1,NC_009480.1_1,16,NZ_CP048048.1,133457,42,150M,capsici_1106
2,NC_009478.1_1,0,NZ_CP048048.1,16945,38,150M,capsici_1106
3,NC_009479.1_1,16,NZ_CP048048.1,42991,42,150M,capsici_1106
4,NZ_CP048047.1_1,0,NZ_CP048047.1,27280,42,150M,capsici_1106
...,...,...,...,...,...,...,...
4397,NC_009478.1_993,0,NC_009479.1,12053,42,150M,michi_contigs
4398,NC_009479.1_993,0,NC_009479.1,1952,0,122M1D28M,michi_contigs
4399,NZ_CP048049.1_993,0,NC_009480.1,2365649,3,150M,michi_contigs
4400,NC_009480.1_994,0,NC_009480.1,81133,42,150M,michi_contigs


In [25]:
df.to_csv('MapeoReads-organismos28K.csv')

Creamos un diccionario para los organimos, y remplazamos en el dataframe


In [4]:
X=df['Organims'].unique().tolist()
len(X)
dic_Organisms={}
for k in range(len(X)):
    dic_Organisms[X[k]]="X{0}".format(k+1)
dic_Organisms
df2=df.replace({"Organims": dic_Organisms}) 

X2=df2['Organims'].unique().tolist() #lista de genomas 
A=powerset(X2) #conjunto potencia
A=A[1:] #eliminiamos el conjunto vacio
df2

,Qname,Flag,Rname,Pos,MapQ,CIGAR,Organims
0,NC_009478.1_0,0,NZ_CP048047.1,578115,42,150M,X1
1,NC_009480.1_1,16,NZ_CP048048.1,133457,42,150M,X1
2,NC_009478.1_1,0,NZ_CP048048.1,16945,38,150M,X1
3,NC_009479.1_1,16,NZ_CP048048.1,42991,42,150M,X1
4,NZ_CP048047.1_1,0,NZ_CP048047.1,27280,42,150M,X1
...,...,...,...,...,...,...,...
4397,NC_009478.1_993,0,NC_009479.1,12053,42,150M,X3
4398,NC_009479.1_993,0,NC_009479.1,1952,0,122M1D28M,X3
4399,NZ_CP048049.1_993,0,NC_009480.1,2365649,3,150M,X3
4400,NC_009480.1_994,0,NC_009480.1,81133,42,150M,X3


In [6]:
B=list(A)

In [5]:
A

[['X1'],
 ['X2'],
 ['X1', 'X2'],
 ['X3'],
 ['X1', 'X3'],
 ['X2', 'X3'],
 ['X1', 'X2', 'X3']]

In [18]:
df2.to_csv('MapeoReads-organismos28K2.csv')

Vamos a crear un diccionario `dic` que contenga los reads y los organismos a los cuales son mapeados

In [6]:
dic={}
l=0
Reads=df2['Qname'].unique().tolist()
for k in Reads:
    valores=list(df2.loc[df2.Qname==k].iloc[:,6])
    dic[k]=valores


In [40]:
list(df2.loc[df2.Qname=="NC_009480.1_1"].iloc[:,6])

['X1', 'X2']

In [15]:
A

[['X1'],
 ['X2'],
 ['X1', 'X2'],
 ['X3'],
 ['X1', 'X3'],
 ['X2', 'X3'],
 ['X1', 'X2', 'X3']]

La función ´Sy´ calcula los reads que son mapeados solamente a $Y\in2^X$.


In [26]:
def Sy(dic,X):
    dict_items=dic.items()
    i=1
    for k in X:
        myValue=k
        globals()["Y{}".format(i)] =[key for key,value in dict_items if value==myValue]
        mykey=[key for key,value in dict_items if value==myValue]
        i+=1
    return Y1,Y2,Y3,Y4,Y5,Y6,Y7
    
    

In [21]:
len(Sy(dic,A)[0])

44

In [20]:
A[0]

['X1']

In [35]:
temp=[key for key,value in dic.items() if value==A[6]]
len(temp)

2203

In [36]:
A

[['X1'],
 ['X2'],
 ['X1', 'X2'],
 ['X3'],
 ['X1', 'X3'],
 ['X2', 'X3'],
 ['X1', 'X2', 'X3']]

In [28]:
# Calculamos SY para cada Y en 2**X
SY=[]
SY= Sy(dic,A)



In [29]:
#Exportamos Sy a txt
for k in range(len(SY)):
    with open('output-tda/prueba1Kread/SyReads/Sy{0}.txt'.format(k), 'w') as f:
        for line in SY[k]:
            f.write(f"{line}\n")

In [14]:
%%bash
for infile in output-tda/prueba1Kread/sorted/*.sam
do 
	base=$(basename ${infile} .sam)
    head -4 ${infile} > output-tda/prueba1Kread/sam_extract/${base}-S_y2.bam  #get the head the .sam file's
	samtools view ${infile} | grep -f output-tda/prueba1Kread/SyReads/S_y2.txt >> output-tda/prueba1Kread/sam_extract/${base}-S_y2.bam # append the read selected 
    done

Vamos a extrar de los archivos *.sam* los read que mapean a $Y$, $ \forall Y \in 2^X $. Utilizamos el script  02-extract-reads-for-SY.sh ( sigue a continuacion), dando un archivo txt con los read el identificador de los read que estan en $Y$, extrae dichos reads del archivo *.sam* para cada genoma de refencia.

In [33]:
cat 02-extract-reads-for-SY.sh


file=$1
#This script will run over all SY to extraxct all reads from *sam file's
base2=$(basename ${file} .txt)
for infile in output-tda/prueba1Kread/sorted/*.sam
do 
	base=$(basename ${infile} .sam)
	head -4 ${infile} > output-tda/prueba1Kread/sam_extract/${base}-${base2}.bam  #get the head the .sam file's
	samtools view ${infile} | grep -f ${file} >> output-tda/prueba1Kread/sam_extract/${base}-${base2}.bam # append the read selected 
	done


In [74]:
%%bash 
for line in output-tda/prueba1Kread/SyReads/*.txt 
do
    bash 02-extract-reads-for-SY.sh $line
done

Procedemos a calcular las profuncidades con `samtools depth`.


In [75]:
%%bash 
for line in output-tda/prueba1Kread/sam_extract/*.bam
do
    base=$(basename ${line} .bam)
    samtools depth ${line} > output-tda/prueba1Kread/depth/${base}.txt
done

In [76]:
A

[['X1'],
 ['X2'],
 ['X1', 'X2'],
 ['X3'],
 ['X1', 'X3'],
 ['X2', 'X3'],
 ['X1', 'X2', 'X3']]

In [ ]:
dic_PX={}
for k in range(len(B)):
    dic_PX[B[k]]="Y{0}".format(k)
dic_PX

In [31]:
A

[['X1'],
 ['X2'],
 ['X1', 'X2'],
 ['X3'],
 ['X1', 'X3'],
 ['X2', 'X3'],
 ['X1', 'X2', 'X3']]

In [10]:

ruta_depth = '/home/shaday/GIT/clavibacter/scripts-tda/output-tda/prueba1Kread/depth/'
contenido_depth = os.listdir(ruta_depth)
contenido_depth
#d1 = pd.read_csv(ruta_depth + 'Sy1-Simulados-subsp_capsici_1101_sorted.txt', sep='\t', header=None) #importar las tablas para cada genoma
#d2 = pd.read_csv(ruta_depth + 'Sy1-Simulados-subsp_capsici_1106_sorted.txt', sep='\t', header=None)
#d3 = pd.read_csv(ruta_depth + 'Sy1-Simulados-subsp_michi_contigs_sorted.txt', sep='\t', header=None) 
#a1=d1[2].sum()
#a2=d1[2].sum()
#a3=d1[2].sum()
#ay=a1+a2+a3
#l1=len(d1)
#l2=len(d2)
#l3=len(d3)
#m=1
#dyx1=ay/(l1)*1/m
#dyx2=ay/(l2)*1/m
#dyx3=ay/(l3)*1/m

['Sy1-Simulados-subsp_capsici_1106_sorted.txt',
 'Sy3-Simulados-subsp_capsici_1101_sorted.txt',
 'Sy4-Simulados-subsp_michi_contigs_sorted.txt',
 'Sy6-Simulados-subsp_michi_contigs_sorted.txt',
 'Sy1-Simulados-subsp_capsici_1101_sorted.txt',
 'Sy0-Simulados-subsp_michi_contigs_sorted.txt',
 'Sy6-Simulados-subsp_capsici_1101_sorted.txt',
 'Sy3-Simulados-subsp_capsici_1106_sorted.txt',
 'Sy2-Simulados-subsp_michi_contigs_sorted.txt',
 '.bam.txt',
 'Sy1-Simulados-subsp_michi_contigs_sorted.txt',
 'Sy2-Simulados-subsp_capsici_1106_sorted.txt',
 'Sy5-Simulados-subsp_capsici_1106_sorted.txt',
 'Sy0-Simulados-subsp_capsici_1106_sorted.txt',
 'Sy4-Simulados-subsp_capsici_1106_sorted.txt',
 'Sy5-Simulados-subsp_capsici_1101_sorted.txt',
 'Sy2-Simulados-subsp_capsici_1101_sorted.txt',
 'Sy4-Simulados-subsp_capsici_1101_sorted.txt',
 'Sy3-Simulados-subsp_michi_contigs_sorted.txt',
 'Sy5-Simulados-subsp_michi_contigs_sorted.txt',
 'Sy0-Simulados-subsp_capsici_1101_sorted.txt',
 'Sy6-Simulados-subs

La funcion `depth_Y` calcula las profundiades  de $Y$ en cada organismo $X_i$

In [33]:
def depth_Y(conjunto,A):
    d1 = pd.read_csv(ruta_depth + conjunto+ '-Simulados-subsp_capsici_1101_sorted.txt', sep='\t', header=None) #importar las tablas para cada genoma
    d2 = pd.read_csv(ruta_depth + conjunto+ '-Simulados-subsp_capsici_1106_sorted.txt', sep='\t', header=None)
    d3 = pd.read_csv(ruta_depth + conjunto+ '-Simulados-subsp_michi_contigs_sorted.txt', sep='\t', header=None) 
    a1=d1[2].sum()
    a2=d1[2].sum()
    a3=d1[2].sum()
    ay=(a1+a2+a3)/3
    l1=len(d1)
    l2=len(d2)
    l3=len(d3)
    m=len(A)
    dyx1=ay/(l1)*1/m**2
    dyx2=ay/(l2)*1/m**2
    dyx3=ay/(l3)*1/m**2
    return dyx1,dyx2,dyx3


In [28]:
depth_Y('Sy2',A[2])

(0.3042449169641471,
 0.3024077134986226,
 0.2695157976610898,
 274435,
 274435,
 274435)

Calculamos $d_{Y}(X_i)$ para cada $X_i \in X$ y cada $Y\in 2^X$

In [30]:
dep=[]
for i in range(len(A)):
    globals()["Dep_Y{}".format(i)]=depth_Y('Sy{0}'.format(i),A[i])
    dep.append(depth_Y('Sy{0}'.format(i),A[i]))


In [31]:
dp = pd.DataFrame(dep,columns=dic_Organisms.values())
dp['Y']=A
dp.to_csv('Depth28K.csv')

In [32]:
dp

,X1,X2,X3,Y
0,1.125988,2.026912,3.533898,[X1]
1,1.484015,1.487425,2.115324,[X2]
2,0.304245,0.302408,0.269516,"[X1, X2]"
3,1.028524,0.776666,0.819433,[X3]
4,0.255431,0.406582,0.300082,"[X1, X3]"
5,0.327746,0.328228,0.363356,"[X2, X3]"
6,0.145665,0.145879,0.161491,"[X1, X2, X3]"


## Simulaciones de reads de Clavibacter a partir de 1, 2 y 3 especies.
utilizaremos los organismos que tenemos en la carpeta `clavi_select`.

In [28]:
#os.makedirs('output-tda/3TP')
#os.makedirs('output-tda/3TP/index-bowtie')
#os.makedirs('output-tda/3TP/out-bowtie')
#os.makedirs('output-tda/3TP/bam')
#os.makedirs('output-tda/3TP/sorted')
#os.makedirs('output-tda/3TP/tablas')
#os.makedirs('output-tda/3TP/SyReads')
#os.makedirs('output-tda/3TP/sam_extract')
os.makedirs('output-tda/3TP/depth')

In [ ]:
%%bash 
source /opt/anaconda3/etc/profile.d/conda.sh
conda activate metagenomics
ls clavi_select/3TP/*fna | while read line; 
do 
    id=$(echo $line| rev| cut -d'_' -f1,2,3 |rev |cut -d'.' -f1);
    bowtie2-build $line output-tda/3TP/index-bowtie/$id;
    echo "$id" >>list-genomasrefencia3TP.txt
done

In [ ]:
%%bash 
source /opt/anaconda3/etc/profile.d/conda.sh
conda activate metagenomics

cat list-genomasrefencia3TP.txt | while read line;
do
bowtie2 -x output-tda/3TP/index-bowtie/$line -U ReadsSimulados/reads-3TP.fastq --no-unal -p 12 -S output-tda/3TP/out-bowtie/Simulados-${line}.sam
done

`6000 reads; of these:
  6000 (100.00%) were unpaired; of these:
    2649 (44.15%) aligned 0 times
    3300 (55.00%) aligned exactly 1 time
    51 (0.85%) aligned >1 times
55.85% overall alignment rate
6000 reads; of these:
  6000 (100.00%) were unpaired; of these:
    1589 (26.48%) aligned 0 times
    4178 (69.63%) aligned exactly 1 time
    233 (3.88%) aligned >1 times
73.52% overall alignment rate
6000 reads; of these:
  6000 (100.00%) were unpaired; of these:
    3733 (62.22%) aligned 0 times
    2248 (37.47%) aligned exactly 1 time
    19 (0.32%) aligned >1 times
37.78% overall alignment rate`

In [13]:
%%bash
for infile in output-tda/3TP/out-bowtie/*.sam
do 
	base=$(basename ${infile} .sam)
	samtools view ${infile} -o output-tda/3TP/bam/${base}.bam
    samtools sort ${infile} -o output-tda/3TP/sorted/${base}_sorted.sam
done

In [14]:
%%bash
for infile in output-tda/3TP/bam/*.bam
do 
	base=$(basename ${infile} .bam)
	samtools sort ${infile} -o output-tda/3TP/sorted/${base}_sorted.bam
	samtools view ${infile} > output-tda/3TP/bam/${base}.txt
done

In [16]:
%%bash
cd ~/GIT/clavibacter/scripts-tda/
ls output-tda/3TP/bam/*.txt | while read line
do
    base=$(basename ${line} .txt)
    cut -f 1-6 $line > output-tda/3TP/tablas/${base}_extrac.txt
done

Importar los resultados en tablas

In [17]:
ruta_tablas = '/home/shaday/GIT/clavibacter/scripts-tda/output-tda/3TP/tablas/'
contenido = os.listdir(ruta_tablas)
genomes = []
df=[]
for i in contenido:
    dftt = pd.read_csv(ruta_tablas + i, sep='\t', header=None) #importar las tablas para cada genoma
    dftt['Organims']=i[16:-11] #agregar el nombre del organismo
    df.append(dftt) #agregar al dataframe
    genomes.append(i[16:-11])
    
df=pd.concat(df)
df.columns= ['Qname','Flag', 'Rname', 'Pos','MapQ','CIGAR','Organims'] #cambiar nombre a las columnas del dataframe
df 

,Qname,Flag,Rname,Pos,MapQ,CIGAR,Organims
0,NC_009480.1_0,0,NZ_CP048049.1,751936,42,150M,capsici_1101
1,NC_009478.1_0,0,NZ_CP048049.1,1337268,42,150M,capsici_1101
2,NC_009479.1_0,0,NZ_CP048049.1,1700224,42,150M,capsici_1101
3,NC_009478.1_1,0,NZ_CP048049.1,2802405,42,150M,capsici_1101
4,NC_009479.1_1,0,NZ_CP048049.1,700488,23,150M,capsici_1101
...,...,...,...,...,...,...,...
4406,NZ_CP048049.1_935,0,NC_009478.1,25732,42,150M,michi_contigs
4407,NZ_CP048050.1_935,0,NC_009480.1,703270,42,150M,michi_contigs
4408,NC_009478.1_935,0,NC_009480.1,395957,8,150M,michi_contigs
4409,NC_009479.1_935,0,NC_009478.1,2855,42,150M,michi_contigs


In [18]:
X=df['Organims'].unique().tolist()
len(X)
dic_Organisms={}
for k in range(len(X)):
    dic_Organisms[X[k]]="X{0}".format(k+1)
dic_Organisms
df2=df.replace({"Organims": dic_Organisms}) 

X2=df2['Organims'].unique().tolist() #lista de genomas 
A=powerset(X2) #conjunto potencia
A=A[1:] #eliminiamos el conjunto vacio
df2

,Qname,Flag,Rname,Pos,MapQ,CIGAR,Organims
0,NC_009480.1_0,0,NZ_CP048049.1,751936,42,150M,X1
1,NC_009478.1_0,0,NZ_CP048049.1,1337268,42,150M,X1
2,NC_009479.1_0,0,NZ_CP048049.1,1700224,42,150M,X1
3,NC_009478.1_1,0,NZ_CP048049.1,2802405,42,150M,X1
4,NC_009479.1_1,0,NZ_CP048049.1,700488,23,150M,X1
...,...,...,...,...,...,...,...
4406,NZ_CP048049.1_935,0,NC_009478.1,25732,42,150M,X3
4407,NZ_CP048050.1_935,0,NC_009480.1,703270,42,150M,X3
4408,NC_009478.1_935,0,NC_009480.1,395957,8,150M,X3
4409,NC_009479.1_935,0,NC_009478.1,2855,42,150M,X3


In [19]:
B=list(A)

In [43]:
df2.to_csv('MapeoReads-3TP.csv')

In [20]:
dic={}
l=0
Reads=df2['Qname'].unique().tolist()
for k in Reads:
    valores=list(df2.loc[df2.Qname==k].iloc[:,6])
    dic[k]=valores


In [23]:
# Calculamos SY para cada Y en 2**X
SY=[]
SY= Sy(dic,A)


In [25]:
#Exportamos Sy a txt
for k in range(len(SY)):
    with open('output-tda/3TP/SyReads/Sy{0}.txt'.format(k), 'w') as f:
        for line in SY[k]:
            f.write(f"{line}\n")

Extraer los reads por SY

In [31]:
%%bash 
for line in output-tda/3TP/SyReads/*.txt 
do
    bash 02-extract-reads-for-SY.sh $line
done

Calcular profundidades

In [32]:
%%bash 
for line in output-tda/3TP/sam_extract/*.bam
do
    base=$(basename ${line} .bam)
    samtools depth ${line} > output-tda/3TP/depth/${base}.txt
done

In [36]:
ruta_depth = '/home/shaday/GIT/clavibacter/scripts-tda/output-tda/3TP/depth/'
contenido_depth = os.listdir(ruta_depth)
contenido_depth

['Sy3-Simulados-subsp_capsici_1101_sorted.txt',
 'Sy4-Simulados-subsp_michi_contigs_sorted.txt',
 'Sy6-Simulados-subsp_michi_contigs_sorted.txt',
 'Sy5-Simulados-subsp_nebraskensis_61-1_sorted.txt',
 'Sy2-Simulados-subsp_nebraskensis_61-1_sorted.txt',
 'Sy1-Simulados-subsp_capsici_1101_sorted.txt',
 'Sy0-Simulados-subsp_michi_contigs_sorted.txt',
 'Sy6-Simulados-subsp_capsici_1101_sorted.txt',
 'Sy3-Simulados-subsp_nebraskensis_61-1_sorted.txt',
 'Sy2-Simulados-subsp_michi_contigs_sorted.txt',
 'Sy1-Simulados-subsp_michi_contigs_sorted.txt',
 'Sy5-Simulados-subsp_capsici_1101_sorted.txt',
 'Sy2-Simulados-subsp_capsici_1101_sorted.txt',
 'Sy6-Simulados-subsp_nebraskensis_61-1_sorted.txt',
 'Sy4-Simulados-subsp_capsici_1101_sorted.txt',
 'Sy3-Simulados-subsp_michi_contigs_sorted.txt',
 'Sy5-Simulados-subsp_michi_contigs_sorted.txt',
 'Sy0-Simulados-subsp_capsici_1101_sorted.txt',
 'Sy4-Simulados-subsp_nebraskensis_61-1_sorted.txt',
 'Sy0-Simulados-subsp_nebraskensis_61-1_sorted.txt',
 'S

In [39]:
def depth_Y(conjunto,A):
    d1 = pd.read_csv(ruta_depth + conjunto+ '-Simulados-subsp_capsici_1101_sorted.txt', sep='\t', header=None) #importar las tablas para cada genoma
    d2 = pd.read_csv(ruta_depth + conjunto+ '-Simulados-subsp_nebraskensis_61-1_sorted.txt', sep='\t', header=None)
    d3 = pd.read_csv(ruta_depth + conjunto+ '-Simulados-subsp_michi_contigs_sorted.txt', sep='\t', header=None) 
    a1=d1[2].sum()
    a2=d1[2].sum()
    a3=d1[2].sum()
    ay=(a1+a2+a3)/3
    l1=len(d1)
    l2=len(d2)
    l3=len(d3)
    m=len(A)
    dyx1=ay/(l1)*1/m**2
    dyx2=ay/(l2)*1/m**2
    dyx3=ay/(l3)*1/m**2
    return dyx1,dyx2,dyx3


In [40]:
dep=[]
for i in range(len(A)):
    globals()["Dep_Y{}".format(i)]=depth_Y('Sy{0}'.format(i),A[i])
    dep.append(depth_Y('Sy{0}'.format(i),A[i]))


In [42]:
dp = pd.DataFrame(dep,columns=dic_Organisms.values())
dp['Y']=A
dp

,X1,X2,X3,Y
0,1.255743,2.321736,1.524887,[X1]
1,1.033729,0.493859,0.775421,[X2]
2,0.251030,0.278589,0.496996,"[X1, X2]"
3,1.161933,1.516791,0.960688,[X3]
4,0.358720,0.844829,0.306749,"[X1, X3]"
5,0.258939,0.131244,0.099443,"[X2, X3]"
6,0.123029,0.146294,0.116136,"[X1, X2, X3]"


In [44]:
dp.to_csv('Depth3TP.csv')

(['X1'],
 ['X2'],
 ['X1', 'X2'],
 ['X3'],
 ['X1', 'X3'],
 ['X2', 'X3'],
 ['X1', 'X2', 'X3'])